In [2]:
import pandas as pd
import os
from pathlib import Path

In [3]:
ckpt_dir = Path('/mnt/sandbox1/alex.li/wandb/run-17680/files/')
dset = 'halo_rgb_stereo_test_v6_0'

In [5]:
df = pd.read_csv(ckpt_dir /dset / 'output.csv')

In [9]:
df['human_state'] == 'true_negative'

0        True
1        True
2        True
3        True
4        True
         ... 
44191    True
44192    True
44193    True
44194    True
44195    True
Name: human_state, Length: 44196, dtype: bool

In [ ]:
d